In [1]:
!pip install gym_super_mario_bros nes_py


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autorom 0.4.2 requires click, which is not installed.
autorom-accept-rom-license 0.4.2 requires click, which is not installed.


  Using cached gym_super_mario_bros-7.4.0-py3-none-any.whl (199 kB)
  Using cached nes_py-8.2.1-cp39-cp39-win_amd64.whl
  Using cached tqdm-4.64.0-py2.py3-none-any.whl (78 kB)
  Using cached gym-0.24.1-py3-none-any.whl
  Using cached pyglet-1.5.21-py3-none-any.whl (1.1 MB)
  Using cached cloudpickle-2.1.0-py3-none-any.whl (25 kB)
  Using cached gym_notices-0.0.7-py3-none-any.whl (2.7 kB)
  Using cached colorama-0.4.5-py2.py3-none-any.whl (16 kB)


In [2]:
import gym_super_mario_bros
#import the joypad wrapper
from nes_py.wrappers import JoypadSpace
#import the simplified controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [3]:
SIMPLE_MOVEMENT

[['NOOP'],
 ['right'],
 ['right', 'A'],
 ['right', 'B'],
 ['right', 'A', 'B'],
 ['A'],
 ['left']]

In [4]:
# Setup Game
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT) #Simplifies the action space

In [5]:
env.action_space

Discrete(7)

In [6]:
env.observation_space.shape

(240, 256, 3)

In [7]:
# random action mario game
done = True
for step in range(100000):
    if done:
        env.reset()
    state,reward,done,info = env.step(env.action_space.sample())
    env.render()
env.close()

C:\ProgramData\Anaconda3\lib\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))


KeyboardInterrupt: 

In [8]:
env.close()

In [32]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113


In [11]:
!pip install --user stable-baselines3[extra]

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\piyus\\AppData\\Roaming\\Python\\Python39\\site-packages\\dateutil\\rrule.py'
Check the permissions.




  Using cached stable_baselines3-1.6.0-py3-none-any.whl (177 kB)
  Using cached gym-0.21.0-py3-none-any.whl
     ---------------------------------------- 10.6/10.6 MB 8.3 MB/s eta 0:00:00
  Using cached matplotlib-3.5.2-cp39-cp39-win_amd64.whl (7.2 MB)
  Using cached Pillow-9.2.0-cp39-cp39-win_amd64.whl (3.3 MB)
  Using cached opencv_python-4.6.0.66-cp36-abi3-win_amd64.whl (35.6 MB)
     ------------------------------------- 904.7/904.7 kB 14.2 MB/s eta 0:00:00
  Using cached psutil-5.9.1-cp39-cp39-win_amd64.whl (245 kB)
  Using cached importlib_resources-5.8.0-py3-none-any.whl (28 kB)
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Using cached fonttools-4.34.4-py3-none-any.whl (944 kB)
  Using cached kiwisolver-1.4.3-cp39-cp39-win_amd64.whl (55 kB)
  Using cached pytz-2022.1-py2.py3-none-any.whl (503 kB)


# Preprocess Environment

In [12]:
# Import frame stacker wrapper and grayscaling wrapper
from gym.wrappers import  GrayScaleObservation
# Import Vectorization wrapper
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
# Import matplotlib
from matplotlib import pyplot as plt

In [13]:
# Setup Game
env = gym_super_mario_bros.make('SuperMarioBros-v0')
#Simplifies the action space
env = JoypadSpace(env, SIMPLE_MOVEMENT)
#1. GrayScale
env = GrayScaleObservation(env,keep_dim=True)
#2. Wrap in dummy environment
env = DummyVecEnv([lambda: env])
#3. Stack the frames
env = VecFrameStack(env, 4, channels_order='last')

In [14]:
state = env.reset()

In [15]:
state.shape

(1, 240, 256, 4)

# 2.Train RL Model

In [18]:
import os
#import ppo for algos
from stable_baselines3 import PPO
#import base callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [19]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [20]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [21]:
callback = TrainAndLoggingCallback(check_freq=100000, save_path=CHECKPOINT_DIR)


In [27]:
model = PPO('CnnPolicy',env,verbose=1,tensorboard_log=LOG_DIR,learning_rate=0.000001,n_steps=512)

Using cpu device
Wrapping the env in a VecTransposeImage.


In [1]:
import torch
torch.cuda.is_available()



False

In [39]:
!pip install pycuda

     ---------------------------------------- 1.7/1.7 MB 9.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 78.6/78.6 kB 4.3 MB/s eta 0:00:00
     ---------------------------------------- 70.3/70.3 kB ? eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pycuda: filename=pycuda-2022.1-cp39-cp39-win_amd64.whl size=367442 sha256=938b4cb553d0abd26b2c451be2cb49479bda4d81620a1625d3834d80989ce25d
  Stored in directory: c:\users\piyus\appdata\local\pip\cache\wheels\58\86\2e\f57c979b96759028b65c201fca38832ef3494e668d456c26fa
  Created wheel for pytools: filename=pytools-202

In [ ]:
model.learn(total_timesteps=1000000, callback=callback)

Logging to ./logs/PPO_1
----------------------------
| time/              |     |
|    fps             | 41  |
|    iterations      | 1   |
|    time_elapsed    | 12  |
|    total_timesteps | 512 |
----------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 9             |
|    iterations           | 2             |
|    time_elapsed         | 106           |
|    total_timesteps      | 1024          |
| train/                  |               |
|    approx_kl            | 2.8905692e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.95         |
|    explained_variance   | -0.00378      |
|    learning_rate        | 1e-06         |
|    loss                 | 115           |
|    n_updates            | 10            |
|    policy_gradient_loss | -0.000272     |
|    value_loss           | 330           |
-------------------------

-------------------------------------------
| time/                   |               |
|    fps                  | 5             |
|    iterations           | 13            |
|    time_elapsed         | 1238          |
|    total_timesteps      | 6656          |
| train/                  |               |
|    approx_kl            | 2.4326262e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.95         |
|    explained_variance   | 0.0111        |
|    learning_rate        | 1e-06         |
|    loss                 | 0.062         |
|    n_updates            | 120           |
|    policy_gradient_loss | -0.000478     |
|    value_loss           | 0.224         |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 5             |
|    iterations           | 14            |
|    time_elapsed         | 1364

In [31]:
torch.cuda.current_device()

AssertionError: Torch not compiled with CUDA enabled